**Data Exploration**
---




In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/3-hatespeech dataset _ nuha_ Are they Our Brothers paper  - Sheet1.xlsx', 
                   index_col = 0
                  )
data.columns = ['label', 'text']
data.head()

,label,text
id,,
930000000000000000,not-hate,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...
930000000000000000,not-hate,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...
930000000000000000,hate,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه
930000000000000000,hate,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...
930000000000000000,hate,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...


In [ ]:
#data.shape
data.info
#data.describe

<bound method DataFrame.info of                        label                                               text
id                                                                             
930000000000000000  not-hate  مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...
930000000000000000  not-hate  مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...
930000000000000000      hate      أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه
930000000000000000      hate  ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...
930000000000000000      hate  عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...
...                      ...                                                ...
958342083311435777      hate  سِجِسْتان:\n.\nقال ياقوت (ت٦٢٦هـ) عنها :\n"ناح...
958341695892017154  not-hate  حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...
958341568469094400      hate  العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...
958339664036945922  not-hate  اتَّبِعِ الْحَقَّ وَلَا تُبَالِ ، وَاحْبِبِ ال...
95833958

# Data Cleaning
---








1.   Remove punctuation marks by using Regex.


In [ ]:
import re,string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', str(text))
    return text_nopunct
data['Text_Without_punct'] = data['text'].apply(lambda x: remove_punct(x))

2.   tokenize the comments by using NLTK’s word_tokenize


In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in data.Text_Without_punct]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



3.  lower case the data



In [ ]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

4. Remove stop words from data using NLTK’s stopwords.

In [ ]:
 nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('arabic')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data

,label,text,Text_Without_punct,Text_Final,tokens
id,,,,,
930000000000000000,not-hate,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...,مؤسسة أرشيف المغرب تتسلم وثائق عن ذاكرة اليهود...,مؤسسة أرشيف المغرب تتسلم وثائق ذاكرة اليهود ال...,"[مؤسسة, أرشيف, المغرب, تتسلم, وثائق, ذاكرة, ال..."
930000000000000000,not-hate,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,مفتي السعودية حماس إرهابية وقتال اليهود حرام ش...,"[مفتي, السعودية, حماس, إرهابية, وقتال, اليهود,..."
930000000000000000,hate,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه,أمراء ال سعود اليهود يخوضون حربا عن الصهيونيه,أمراء ال سعود اليهود يخوضون حربا الصهيونيه,"[أمراء, ال, سعود, اليهود, يخوضون, حربا, الصهيو..."
930000000000000000,hate,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,ياحكام العرب أقسم باالله عذاب الله واقع عليكم ...,"[ياحكام, العرب, أقسم, باالله, عذاب, الله, واقع..."
930000000000000000,hate,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,عندما كنت بأمريكا حتم علي عملي ان احتك بفتيات ...,"[عندما, كنت, بأمريكا, حتم, علي, عملي, ان, احتك..."
...,...,...,...,...,...
958342083311435777,hate,"سِجِسْتان:\n.\nقال ياقوت (ت٦٢٦هـ) عنها :\n""ناح...",سِجِسْتان\n\nقال ياقوت ت٦٢٦هـ عنها \nناحية كبي...,سِجِسْتان قال ياقوت ت٦٢٦هـ عنها ناحية كبيرة وو...,"[سِجِسْتان, قال, ياقوت, ت٦٢٦هـ, عنها, ناحية, ك..."
958341695892017154,not-hate,حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...,حسبي الله لا إله إلا هو عليه توكلت وهو ربّ الع...,حسبي الله إله توكلت ربّ العرش العظيممن قالها ي...,"[حسبي, الله, إله, توكلت, ربّ, العرش, العظيممن,..."
958341568469094400,hate,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,العرب البقرة الحلوب للمستعمر الغربي الصليبي وا...,"[العرب, البقرة, الحلوب, للمستعمر, الغربي, الصل..."


# Splitting Data into Test and Train


In [ ]:
#Now we split our data set into train and test. We will use 90 % data for training and 10 % for testing.
!pip install scikit-learn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X = data.text.astype(str)
Y = data.label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
#data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

# Process the data¶

*   Tokenize the data and convert the text to sequences.
*   Add padding to ensure that all the sequences have the same shape.
*   There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.






In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
import numpy as np

max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

# RNN

In [ ]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from keras.optimizers import RMSprop

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
#Call the function and compile the model.
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [ ]:
#Fit on the training data.
from keras.callbacks import EarlyStopping

model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
26/26 [==============================] - 5s 178ms/step - loss: 0.6882 - accuracy: 0.5497 - val_loss: 0.6840 - val_accuracy: 0.5468
Epoch 2/10
26/26 [==============================] - 4s 159ms/step - loss: 0.6578 - accuracy: 0.5971 - val_loss: 0.6637 - val_accuracy: 0.5893
Epoch 3/10
26/26 [==============================] - 4s 157ms/step - loss: 0.6103 - accuracy: 0.6750 - val_loss: 0.6847 - val_accuracy: 0.5857


In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,y_test)


64/64 [==============================] - 1s 15ms/step - loss: 0.6516 - accuracy: 0.6249


In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.652
  Accuracy: 0.625
